In [1]:
!pip install tweepy

In [8]:
###############################################################################
# NLP 2021-I
# Clase 12: Mayo 13, 2021
###############################################################################

# Primer link: https://developer.twitter.com/en/portal/projects-and-app
import pandas as pd
import tweepy

# Leer las llaves
keys = pd.read_csv("mis_llaves.csv", sep=';',header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [9]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

Documentación: http://docs.tweepy.org/en/latest/

In [ ]:
api.update_status('Aún sigo preparando clase...')

In [10]:
tweets = tweepy.Cursor(api.search,
                       q=['giro de italia -RT'],
                       lang='es',
                       geocode=f'4.7110,-74.0721,100km',
                       tweet_mode='extended',
                       include_rts=False).items(100)

In [11]:
# Objeto generador
t = [tweet for tweet in tweets]

In [12]:
t_processed = []

for tweet in t:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogotá")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t_processed.append(temp)

In [13]:
data = pd.DataFrame(t_processed)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweet,date,city,user_location,id,link
0,Ganó en los últimos metros: Andrea Vendrame se...,2021-05-20 20:49:22,Bogotá,"Bogotá, D.C., Colombia",1395481801188167691,https://twitter.com/i/web/status/1395481801188...
1,Ganó en los últimos metros: Andrea Vendrame se...,2021-05-20 20:48:45,Bogotá,Bogotá - Colombia,1395481645734580225,https://twitter.com/i/web/status/1395481645734...
2,Sin cambios: Colombianos pasaron la montaña de...,2021-05-20 20:48:19,Bogotá,"Bogotá, D.C., Colombia",1395481535193698306,https://twitter.com/i/web/status/1395481535193...
3,Sin cambios: Colombianos pasaron la montaña de...,2021-05-20 20:48:16,Bogotá,Bogotá - Colombia,1395481523667734534,https://twitter.com/i/web/status/1395481523667...
4,Ahora falta que el Comité del Paro diga que Eg...,2021-05-20 20:40:51,Bogotá,Bogotá - Colombia,1395479656980226051,https://twitter.com/i/web/status/1395479656980...


In [16]:
data.to_csv("Archivo.csv", index=False)

In [17]:
!pip install snscrape

In [18]:
import snscrape.modules.twitter as sntwitter

maxTweets = 10  
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#eragoldeyepes ' + 'since:2014-07-15 until:2014-08-15').get_items()) :
        if i > maxTweets :
            break
        print(tweet)
        print(tweet.date)
        print(tweet.content)
        print("*"*100)
        print()

https://twitter.com/CANmialegria/status/500066563522056192
2014-08-14 23:48:54+00:00
Cambiemos el HT #EraGolDeYepes, por #EraGolDeCopete. ¡Nunca hubo fuera de lugar!
¿#LaCulpaEsDePostobon?  @PradoVerde1999 @vamosmiverdecom
****************************************************************************************************

https://twitter.com/cami_canoles/status/500064190288384000
2014-08-14 23:39:28+00:00
http://t.co/X3CpWL1Q3x #eragoldeyepes
****************************************************************************************************

https://twitter.com/TamayoJr11/status/500062617969643520
2014-08-14 23:33:13+00:00
#EraGolDeYepes http://t.co/K9Am0bxOIh
****************************************************************************************************

https://twitter.com/Edw_Chacon/status/500055868361412608
2014-08-14 23:06:24+00:00
JjaJja Todavia Me Acuerdo :') #EraGolDeYepes http://t.co/xFkiku07ae
****************************************************************************

In [19]:
data = data[['tweet', 'date', 'city']]
data.head()

,tweet,date,city
0,Ganó en los últimos metros: Andrea Vendrame se...,2021-05-20 20:49:22,Bogotá
1,Ganó en los últimos metros: Andrea Vendrame se...,2021-05-20 20:48:45,Bogotá
2,Sin cambios: Colombianos pasaron la montaña de...,2021-05-20 20:48:19,Bogotá
3,Sin cambios: Colombianos pasaron la montaña de...,2021-05-20 20:48:16,Bogotá
4,Ahora falta que el Comité del Paro diga que Eg...,2021-05-20 20:40:51,Bogotá


In [20]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [22]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

,tweet,date,city,clean,links,hashtags,mentions
0,Ganó en los últimos metros: Andrea Vendrame se...,2021-05-20 20:49:22,Bogotá,Ganó en los últimos metros: Andrea Vendrame se...,[https://t.co/QJVrQraxkM],[],[]
1,Ganó en los últimos metros: Andrea Vendrame se...,2021-05-20 20:48:45,Bogotá,Ganó en los últimos metros: Andrea Vendrame se...,[https://t.co/nnSAD3ZkRC],[],[]
2,Sin cambios: Colombianos pasaron la montaña de...,2021-05-20 20:48:19,Bogotá,Sin cambios: Colombianos pasaron la montaña de...,[https://t.co/TANWB6rktR],[],[]
3,Sin cambios: Colombianos pasaron la montaña de...,2021-05-20 20:48:16,Bogotá,Sin cambios: Colombianos pasaron la montaña de...,[https://t.co/yBNY5BtqdT],[],[]
4,Ahora falta que el Comité del Paro diga que Eg...,2021-05-20 20:40:51,Bogotá,Ahora falta que el Comité del Paro diga que Eg...,[],[],[]


In [23]:
!pip install regex pip install emoji

In [24]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list

data['emojis'] = data['clean'].apply(lambda text: get_emojis(text))

In [25]:
data.head()

,tweet,date,city,clean,links,hashtags,mentions,emojis
0,Ganó en los últimos metros: Andrea Vendrame se...,2021-05-20 20:49:22,Bogotá,Ganó en los últimos metros: Andrea Vendrame se...,[https://t.co/QJVrQraxkM],[],[],[]
1,Ganó en los últimos metros: Andrea Vendrame se...,2021-05-20 20:48:45,Bogotá,Ganó en los últimos metros: Andrea Vendrame se...,[https://t.co/nnSAD3ZkRC],[],[],[]
2,Sin cambios: Colombianos pasaron la montaña de...,2021-05-20 20:48:19,Bogotá,Sin cambios: Colombianos pasaron la montaña de...,[https://t.co/TANWB6rktR],[],[],[]
3,Sin cambios: Colombianos pasaron la montaña de...,2021-05-20 20:48:16,Bogotá,Sin cambios: Colombianos pasaron la montaña de...,[https://t.co/yBNY5BtqdT],[],[],[]
4,Ahora falta que el Comité del Paro diga que Eg...,2021-05-20 20:40:51,Bogotá,Ahora falta que el Comité del Paro diga que Eg...,[],[],[],[]


In [26]:
!pip install plotly

In [27]:
'''
data['fecha'] = data.date.dt.floor('T').dt.time

temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp
'''

"\ndata['fecha'] = data.date.dt.floor('T').dt.time\n\ntemp = pd.DataFrame(data.fecha.value_counts()).reset_index()\ntemp.columns = ['fecha', 'cnt']\ntemp = temp.sort_values('fecha')\ntemp\n"

In [28]:
data['fecha'] = data.date.dt.date
temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp

,fecha,cnt
0,2021-05-20,100


In [29]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [30]:
'''
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'#1DA1F2'})

layout = go.Layout(title='Número de tuits por día')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)
'''

'\ntrace = go.Scatter(x=temp.fecha.values,\n                   y=temp.cnt.values,\n                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],\n                   hoverinfo=\'text\',\n                   mode=\'lines+markers\',\n                   name=\'Días\',\n                   line={\'color\':\'#1DA1F2\'})\n\nlayout = go.Layout(title=\'Número de tuits por día\')\n\nfig = go.Figure(data=[trace], layout=layout)\niplot(fig)\n'

In [31]:
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'#1DA1F2'})

layout = go.Layout(title='Número de tuits por día')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [32]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(),Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)
temp_emojis.head()

,emoji,cnt
8,🚴‍♂️,10
15,💗,7
1,🚴,3
12,🏆,2
3,⬇,2


In [33]:
trace = go.Bar(x=temp_emojis.emoji.values[:15],
               y=temp_emojis.cnt.values[:15])

layout = go.Layout(title='Emojis más usados')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)